In [4]:
import torch
import torch.nn as nn
from torch.autograd import Variable

import numpy as np
import string
from tqdm import tqdm_notebook

all_letters = string.ascii_letters + " .,;'"

def read_data():
    data_path = 'data/HPserie/HPSerie'
    with open(data_path, 'r') as myfile:
        data=myfile.read().replace('\n', '')

    return data

def seq_gen(data, seq_len):
    total_size = len(data)
    ex_len = seq_len + 1
    idxs = list(range(0, total_size, ex_len))
    np.random.shuffle(idxs)
    yield len(idxs)
    while True:
        for idx in idxs:
            d = data[idx:idx + ex_len]
            X = d[0:-1]
            Y = d[1:]
            X_txt = ''.join(X)
            Y_txt = ''.join(Y)
            X_encoded = Variable(torch.Tensor([char_to_idx[x] for x in X]).long().cuda())
            Y_encoded = Variable(torch.Tensor([char_to_idx[x] for x in Y]).long().cuda())
            
            yield X_txt, Y_txt, X_encoded, Y_encoded
        np.random.shuffle(idxs)
    

data = read_data()
data = [x for x in data if x in all_letters]


char_to_idx = {x: i for i,x in enumerate(all_letters)}
idx_to_char = {v: k for k,v in char_to_idx.items()}

In [5]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers

        self.encoder = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)

    def forward(self, input, hidden):
        input = self.encoder(input.view(1, -1))
        output, hidden = self.gru(input.view(1, 1, -1), hidden)
        output = self.decoder(output.view(1, -1))
        return output, hidden

    def init_hidden(self):
        return Variable(torch.zeros(self.n_layers, 1, self.hidden_size).cuda())

In [6]:
input_size = len(all_letters)
hidden_size = 100
n_layers = 1

print_every = 50 
plot_every = 100
epochs = 2


model = RNN(input_size, hidden_size, input_size, n_layers)
model_optimizer = torch.optim.Adam(model.parameters())

model.cuda()

RNN (
  (encoder): Embedding(57, 100)
  (gru): GRU(100, 100)
  (decoder): Linear (100 -> 57)
)

In [7]:
criterion = nn.CrossEntropyLoss()

def train(inp, target):
    hidden = model.init_hidden()
    model.zero_grad()
    loss = 0

    for c in range(len(inp)):
        output, hidden = model(inp[c], hidden)
        loss += criterion(output, target[c])

    loss.backward()
    model_optimizer.step()

    return loss.data[0] / float(len(inp))

In [10]:
G = seq_gen(data, 200)
batch_per_epoch = next(G)

loss_avg = 0
losses = []

for epoch in range(1, epochs+1):
    bar = tqdm_notebook(range(batch_per_epoch))
    for b in bar:
        X_txt, Y_txt, X_enc, Y_enc = next(G)
        loss = train(X_enc, Y_enc)       
        loss_avg += loss
        
        bar.set_postfix(loss=loss) 
        if b % plot_every == 100:
            losses.append(loss_avg / plot_every)
            loss_avg = 0
    

KeyboardInterrupt: 

In [ ]:
torch.cuda.is_available()